In [16]:
## import packages
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from util import createGrps
import sys


In [29]:
## First get the groups
# parameters for group generation
size = 20
grpNum = 20
seed = 12345

# load user data
user_data = pd.read_csv("users.tsv", sep='\t')

# generate synthetic groups of users and display them
groups2 = createGrps(user_data, size, grpNum)
display(groups2)
print()

,Users
0,"[1035974.0, 344708.0, 1066899.0, 289064.0, 648..."
1,"[75923.0, 23973.0, 761000.0, 283383.0, 234247...."
2,"[1395023.0, 1405473.0, 363116.0, 1315134.0, 24..."
3,"[1305595.0, 1055885.0, 1083613.0, 1051390.0, 1..."
4,"[178791.0, 821044.0, 788454.0, 382702.0, 31704..."
5,"[100131.0, 234749.0, 608257.0, 188775.0, 30679..."
6,"[1102662.0, 756498.0, 998930.0, 271617.0, 9393..."
7,"[766070.0, 1468060.0, 1083803.0, 1086288.0, 96..."
8,"[1187420.0, 784738.0, 424114.0, 1268900.0, 506..."
9,"[1082996.0, 678322.0, 401094.0, 1309611.0, 390..."


In [18]:
## Get the training data and train the user-user collaborative filterring
# read the data (about applications)
data = pd.read_csv('user_with_negative_ratings_full.csv', delimiter=',')

# construct dataframe in format (user, item, rating) via column addition
df_ui = data.rename(columns={"UserID": "user", "JobID": "item", "Rating":"rating"})
# check data being read properly
print(df_ui.head(10)) 

# train UserUser collaborative filterring
user_user = UserUser(10, min_nbrs=3)  # Minimum (3) and maximum (10) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

   user    item  rating
0    47  169528       1
1    47  284009       1
2    47    2121       1
3    47  848187       1
4    47  733748       1
5    47  576958       1
6    47  262470       1
7    47  602298       1
8    47  982331       0
9    47  979937       0


In [28]:
## Create a User-Item matrix of scores so we can apply one of the aggregation strategies
# iterate through groups generated
for i, row in groups2.iterrows():
    # get the array of users from the row
    synGrp = row[0]
    # create empty dataframe for ratings of Users to Items
    ratings_grp = pd.DataFrame(columns=["UserID", "item", "score"])
    # create an empty dataframe to store unique jobs of this syn group
    items_grp = pd.DataFrame(columns=["item"])
    # iterate through each user and generate top 10 recommendations
    db_ind = 0
    for userId in synGrp:
        # generate top 10: (item, score)
        top10 = recsys.recommend(userId, 10)
        # concatenate and drop duplicates to store the jobs
        items_grp = (pd.concat([items_grp, top10[["item"]]], axis=0, ignore_index=True)).drop_duplicates()
        # add top 10 with associated user ID
        top10.insert(0, "UserID", np.full((len(top10), 1), userId), True)
        ratings_grp = pd.concat()
        if (db_ind==2): sys.exit()
        db_ind+=1

/var/folders/vt/47mxbcvj5rd7mmyk053h49m00000gn/T/ipykernel_32257/1468054515.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  synGrp = row[0]


,UserID,item,score
0,1140654.0,844808,1.272176
1,1140654.0,878783,1.146167
2,1140654.0,620921,1.142406
3,1140654.0,58228,1.125161
4,1140654.0,180316,1.096776
5,1140654.0,814510,1.092797
6,1140654.0,607759,1.043662
7,1140654.0,105595,1.019713
8,1140654.0,802824,1.007386
9,1140654.0,1081031,0.985837


user 563685 has no ratings and none provided


,UserID,item,score


,UserID,item,score
0,1223756.0,425361,0.952900
1,1223756.0,971237,0.943182
2,1223756.0,188535,0.838321
3,1223756.0,180819,0.819429
4,1223756.0,751709,0.818571
5,1223756.0,548373,0.807838
6,1223756.0,1115031,0.792775
7,1223756.0,94383,0.787262
8,1223756.0,215968,0.780308
9,1223756.0,482806,0.770938


SystemExit: 

/Users/kirillgugunishvili/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
